In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))
import xgboost as xgb
# Any results you write to the current directory are saved as output.

In [ ]:
import json  
import zipfile  

path= "../input/two-sigma-connect-rental-listing-inquiries/"
d = None  
data = None  

with open(path+"train.json") as f:
    #data = f.read()
    d=json.load(f)
    f.close()

type(d)


In [ ]:
##Test

d_test = None
#data_test=None
with open(path+"test.json") as f1:
    #data = f.read()
    d_test=json.load(f1)
    f1.close()

type(d_test)
df_test=pd.DataFrame(d_test)
len(df_test)

In [ ]:
df=pd.DataFrame(d)
len(df)
df.head(10)

In [ ]:
df['created'] = df['created'].astype('datetime64[ns]')

In [ ]:
a=max(df['created'])-df['created']

In [ ]:
df['daysFromCreated']=a/np.timedelta64(1, 'D')

In [ ]:
df.head()

In [ ]:
xl=df.longitude
yl=df.latitude
xl=(xl-np.mean(xl))/np.std(xl)
yl=(yl-np.mean(yl))/np.std(yl)

df.longitude=xl
df.latitude=yl

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xl,yl,'ro')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
a= np.array(xl)
b=np.array(yl)

In [ ]:
c=np.column_stack((a,b))

In [ ]:
kmeans = KMeans(n_clusters=20)
kmeans.fit_predict(c)

In [ ]:
centers = kmeans.cluster_centers_

In [ ]:
prediction = kmeans.predict(c)

In [ ]:
prediction

In [ ]:
plt.scatter(centers[:, 0], centers[:, 1], marker='x')

plt.show()

In [ ]:
df["KMeans_Clusters"]= prediction

In [ ]:
df.head(50)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.groupby("KMeans_Clusters").count()

In [ ]:
c=[1,2,3,4,5,8,13,14,16,17,18,19]

In [ ]:
df['KMeans_Clusters'].replace(1, 10,inplace=True)
df['KMeans_Clusters'].replace(2, 10,inplace=True)
df['KMeans_Clusters'].replace(3, 10,inplace=True)
df['KMeans_Clusters'].replace(4, 10,inplace=True)
df['KMeans_Clusters'].replace(5, 10,inplace=True)
df['KMeans_Clusters'].replace(8, 10,inplace=True)
df['KMeans_Clusters'].replace(13, 10,inplace=True)
df['KMeans_Clusters'].replace(14, 10,inplace=True)
df['KMeans_Clusters'].replace(16, 10,inplace=True)
df['KMeans_Clusters'].replace(17, 10,inplace=True)
df['KMeans_Clusters'].replace(18, 10,inplace=True)
df['KMeans_Clusters'].replace(19, 10,inplace=True)


In [ ]:
##Test

df_test['created'] = df_test['created'].astype('datetime64[ns]')
#a=max(df_test['created'])-df_test['created']
df_test['daysFromCreated']=(max(df_test['created'])-df_test['created'])/np.timedelta64(1, 'D')
xl_test=df_test.longitude
yl_test=df_test.latitude
xl_test=(xl_test-np.mean(xl))/np.std(xl)
yl_test=(yl_test-np.mean(yl))/np.std(yl)
a_test= np.array(xl_test)
b_test=np.array(yl_test)
c_test=np.column_stack((a_test,b_test))
kmeans.fit_predict(c_test)
prediction_test = kmeans.predict(c_test)
df_test.longitude=xl_test
df_test.latitude=yl_test

df_test["KMeans_Clusters"]= prediction_test

df_test['KMeans_Clusters'].replace(1, 10,inplace=True)
df_test['KMeans_Clusters'].replace(2, 10,inplace=True)
df_test['KMeans_Clusters'].replace(3, 10,inplace=True)
df_test['KMeans_Clusters'].replace(4, 10,inplace=True)
df_test['KMeans_Clusters'].replace(5, 10,inplace=True)
df_test['KMeans_Clusters'].replace(8, 10,inplace=True)
df_test['KMeans_Clusters'].replace(13, 10,inplace=True)
df_test['KMeans_Clusters'].replace(14, 10,inplace=True)
df_test['KMeans_Clusters'].replace(16, 10,inplace=True)
df_test['KMeans_Clusters'].replace(17, 10,inplace=True)
df_test['KMeans_Clusters'].replace(18, 10,inplace=True)
df_test['KMeans_Clusters'].replace(19, 10,inplace=True)
df_test['KMeans_Clusters']=df_test['KMeans_Clusters'].astype(object)

In [ ]:
df['KMeans_Clusters']=df['KMeans_Clusters'].astype(object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer



In [ ]:
df['features'] = df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

In [ ]:
tfidf = CountVectorizer(stop_words='english', max_features=200)


In [ ]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .80

In [ ]:
df.head()

In [ ]:
train, test = df[df['is_train']==True], df[df['is_train']==False]

In [ ]:
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

In [ ]:
x=[0,1,8,10,13,15,16]
features = df.columns[x]

features

In [ ]:
y = pd.factorize(train['interest_level'])[0]
y

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train.shape

In [ ]:
from scipy import sparse
tr_sparse = tfidf.fit_transform(train["features"])
te_sparse = tfidf.fit_transform(test["features"])

In [ ]:
tr_sparse.shape

In [ ]:
categorical = ["KMeans_Clusters"]
for f in categorical:
        if train[f].dtype=='object':
            lbl = LabelEncoder()
            lbl.fit(list(train[f].values) + list(test[f].values))
            train[f] = lbl.transform(list(train[f].values))
            test[f] = lbl.transform(list(test[f].values))
#            features_to_use.append(f)

In [ ]:
train_x = sparse.hstack([train[features], tr_sparse]).tocsr()
train_y= train['interest_level']

In [ ]:
#preds, model = runXGB(train_X, train_y, test_X, num_rounds=2000)
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.02
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = seed_val
num_rounds = num_rounds

plst = list(param.items())
xgtrain = xgb.DMatrix(train_X, label=train_y)

In [ ]:
xgtest = xgb.DMatrix(test_X, label=test_y)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)

In [ ]:
xgtest = xgb.DMatrix(test_X)
model = xgb.train(plst, xgtrain, num_rounds)

In [ ]:
pred_test_y = model.predict(xgtest)

In [ ]:
clf = RandomForestClassifier(n_estimators=450,max_depth=35)

# Train the classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(train_x, train['interest_level'])

In [ ]:
test_x = sparse.hstack([test[features], te_sparse]).tocsr()

In [ ]:
preds=clf.predict(test_x)

In [ ]:
clf.predict_proba(test_x)[0:10]

In [ ]:
preds[0:5]

In [ ]:
test['interest_level'][0:5]

In [ ]:
pd.crosstab(test['interest_level'], preds, rownames=['Actual interest_level'], colnames=['Predicted interest_level'])

In [ ]:
import sklearn

In [ ]:
sklearn.metrics.log_loss(test['interest_level'], clf.predict_proba(test_x))

In [ ]:
##Test

pred_actualTest = clf.predict_proba(df_test[features])


In [ ]:
pred_actualTest

In [ ]:
your_permutation = [0,2,1]
i = np.argsort(your_permutation)
i
pred_actualTest1=pred_actualTest[:,i]

In [ ]:
out_df = pd.DataFrame(pred_actualTest1)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = df_test.listing_id.values

In [ ]:
out_df.head()

In [ ]:
out_df.to_csv("first1.csv", index=False)

In [ ]:
print(check_output(["ls"]).decode("utf8"))

In [ ]:
out_df